In [318]:
"""Calculation of the Kaustik of a Laserbeam."""

import matplotlib.pyplot as plt
import numpy as np
from SecondaryValue import SecondaryValue
from scipy.optimize import curve_fit
import pandas as pd
import glob


In [319]:
# Input Parameters
R = 100  # cm
L = 80  # cm
f = 15  # cm
lam = 0.00006328  # cm
lens_pos = 25+29.5+10 # cm


In [320]:
# Optical Matrix Elements
A = SecondaryValue('1-x/f', defaults=dict(f=f))
B = SecondaryValue('s+x-x*s/f', defaults=dict(f=f))
C = SecondaryValue('-1/f', defaults=dict(f=f))
D = SecondaryValue('1-s/f', defaults=dict(f=f))


In [321]:
w0 = SecondaryValue('((l/pi)^2*L*(R-L))^(1/4)', defaults=dict(l=lam, L=L, R=R, pi=np.pi))

In [322]:
# b parameter
b = SecondaryValue('l/(pi*w0^2)', defaults=dict(l=lam, pi=np.pi), dependencies=dict(w0=w0))
new_b = SecondaryValue('b*(A*D-C*B)/(A^2+B^2*b^2)', dependencies=dict(A=A, B=B, C=C, D=D, b=b))
b_before = SecondaryValue('b/(1+x^2*b^2)', dependencies=dict(b=b))

In [323]:
# omega
w = SecondaryValue('sqrt(l/(p*b))', defaults=dict(l=lam, p=np.pi), dependencies=dict(b=new_b))
w_before = SecondaryValue('sqrt(l/(p*b))', defaults=dict(l=lam, p=np.pi), dependencies=dict(b=b_before))

In [324]:
w0()

0.02838496052328435

In [325]:
data = pd.read_csv(r'./../messungen/Boettcher_Matthes/Kaustik-Messung/11_2008-07-13_03h-00m-41s.csv', sep=';', encoding='latin1')
fwhm = 'Value.18'
zs = np.array([11, 13, 15, 17, 19, 21, 23, 27])

In [326]:
def extract_fwhm(z):
    filename = \
        glob.glob(f'./../messungen/Boettcher_Matthes/Kaustik-Messung/{z}*.csv')[0]
    dat = pd.read_csv(filename, sep=';', encoding='latin1')
    return dat[fwhm][0]


In [327]:
widths = np.array([extract_fwhm(z) for z in zs]) * 0.00056 / (np.sqrt(2*np.log(2))) # cm

In [328]:
popt, pcov = curve_fit(lambda x, w0, off: w(w0=w0, s=lens_pos, x=[x+off]), zs, widths)

In [329]:
%matplotlib qt5
xs_before = np.linspace(0, lens_pos, 1000)
xs_after = np.linspace(0, 40, 1000)
plt.plot(xs_before, w_before(w0=popt[0], x=[xs_before]), color='grey',
         linestyle='--')
plt.plot(lens_pos + xs_after, w(w0=popt[0], s=lens_pos, x=[xs_after]), color='grey',
         linestyle='--', label='Theoretische Kaustik, $w_0={:f}cm$'.format(popt[0]))
plt.errorbar(lens_pos + zs + popt[1], widths, yerr=0.00056,
             xerr=1, linestyle='none', marker='x', markersize=10,
             ecolor='red', elinewidth=1, barsabove=True,
             label='Gemessene Kaustik', zorder=100)
plt.xlabel('z [cm]')
plt.ylabel('w [cm]')
plt.legend()
plt.grid()
plt.show()

In [330]:
np.sqrt(np.diag(pcov))

array([0.00158225, 0.234916  ])

In [331]:
popt

array([0.03963238, 1.20690559])

In [332]:
w0(L=(80,.5))

(0.02838496052328435, 0.0001330545024528954)

In [344]:

print(pd.DataFrame(np.array([zs, np.array([extract_fwhm(z) for z in zs]), np.around(widths*10*1000)]).T).to_csv())

,0,1,2
0,11.0,29.9,142.0
1,13.0,19.1,91.0
2,15.0,14.0,67.0
3,17.0,17.5,83.0
4,19.0,27.7,132.0
5,21.0,41.5,197.0
6,23.0,56.0,266.0
7,27.0,78.3,372.0

